# Trust region policy optimization

- 높은 차원의 action space를 가진 환경부터 action은 적지만 control parameter가 많은 환경까지 범용적으로 사용할 수 있는 좋은 알고리즘이다.

## 서론

- 문제 정의, expected discounted reward를 최대화하는 policy를 찾자!

- MDP: \\(\langle S, A, P, r, \rho_0, \gamma \rangle\\)
  - \\(P \\): state transition probability
  - \\(\rho_0\\): 초기 state \\(s_0\\)에 대한 distribution
  - \\(r\\): reward function
  - \\(\gamma \\): discount factor

### Value based 강화학습

- Q-function으로 action을 선택한다.
- ex) SARSA, Q-Learning, DQN, Dueling network, prioritized experience replay, ...

### Policy based 강화학습

- policy를 가지고 있으며 policy는 parameter로 이루어져 있음.
- REINFORCE, Actor-critic, A3C, TRPO, PPO, GAE, ...

## Policy Gradient

- Objective function(policy parameter \\(\theta\\)의 함수)을 최대화하는 policy를 구하는 것
- 각각의 iteration마다 objective function의 gradient를 따라 paramter를 update한다.

### Policy gradient의 문제

- Sample의 효율성이 없다.
  - policy gradient는 현재 policy에 대한 estimate을 update하고 있는데, 한번 사용하면 버리게됨 따라서 효율성에서 떨어진다.
- parameter space에서 거리와 policy space에서 거리가 다르다.
  - parameter에서 update를 했을 때 policy가 받는 영향이 클수도 있고 작을 수도 있다.

### Importance sampling

- 효율적으로 기대값을 추정하기 위해 만들어졌다.
- 목적, 기대값을 계산하고자 하는 확률 분포 p(x)의 확률 밀도 함수를 알고 있지만 p에서 샘플을 생성하기 어려울 때 비교적 샘플을 생성하기 쉬운 q(x)에서 샘플을 생성하여 p의 기대값을 계산한다.

$$ E_{x\sim p}[f(x)] = \int p(x)f(x)dx = \int \frac{p(x)}{q(x)}q(x)f(x)dx = E_{x \sim q}\begin{bmatrix} \frac{p(x)}{q(x)}f(x) \end{bmatrix} $$

- 위 식에서 \\(\frac{p(x)}{q(x)}\\)를 likelihood ratio라 하며, \\(p\\)를 normal distribution, \\(q\\)를 importance distribution이라 한다.

- 따라서, importance sampling에서 \\(p\\)의 기대값 \\(E_{x \sim p}[f(x)]\\)를 아래와 같이 추정한다.
$$ E_{x \sim p}[f(x)] \approx \frac{1}{N} \sum_{n=1}^N \frac{p(x_n)}{q(x_n)}f(x_n),\; x_n\sim q $$

### policy gradient, importance sampling

- policy gradient에서 이전 policy의 sample을 사용하기 위해서 importance sampling 사용할 수 있음.
- importance weight(\\(\frac{p(x_n)}{q(x_n)}\\))는 unbound(variance의 범위가 커서) -> 학습이 잘 안된다.

### step in policy space

- 앞서 문제로 언급되었던, parameter space는 policy space가 아니다.
  - 그렇다면, parameter space가 아니라 policy space에서 조금씩 update가 가능한가?
    - 두 policy(old policy, new policy)사이에 KL-divergence를 constraint로 (trust region)

#### KL-divergence

- 얼마나 두 확률분포가 다른지를 설명해주는 지표
- \\(D_KL(P||Q) = \sum_i P(i)log \frac{P(i)}{Q(i)} \\)

- 주로 아직 확인되지 않은 모델을 특정 확률분포로 근사시키는데 사용한다.
- KL-divergence를 가장 낮추는 정규분포를 찾는다.

- Q(근사 모델)를 최대한 P(실제 데이터 분포)에 가깝게 만들려면, KL-divergence를 최소화하도록 학습을 시킨다.
- KL-divergence의 특징
  - 항상 0이상의 값을 갖는다.
  - 비대칭적 함수이다.
    - \\(D_KL(p||q)\\)와 \\(D_KL(q||p)\\)는 다른값이다.
  - 두 확률 분포가 동일하면 KL-divergence는 0이 된다.

- KL-divergence 분해하기
  $$ \begin{split} D_KL(P||Q) &= \sum_i P(i) log \frac{P(i)}{Q(i)} \\ &= \sum_i P(i)logP(i) - \sum_i P(i)logQ(i) \\ &= -H(P) + H(P,Q) \end{split} $$

- KL-divergence를 둘로 나눌 수 있기 때문에 크로스 엔트로피로 표현이 가능하다.

### stable한 update는 안되는가?

- lower bound를 최적화 하자!

## TRPO를 공부하는데 참조하였습니다.

[가깝고도 먼 Trpo](https://www.slideshare.net/WoongwonLee/trpo-87165690)